In [ ]:
!uv pip install pyLDAvis

In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.lda_model
import time
from contextlib import contextmanager

@contextmanager
def timer(name="_func_"):
    """Context manager to measure execution time"""
    start_time = time.time()
    print(f"\nstart: {name}")
    try:
        yield
    finally:
        end_time = time.time()
        print(f"end: {name} - exec time: {end_time - start_time:.4f} seconds")

# dataset
with timer("fetch"):
    newsgroups = fetch_20newsgroups(subset='all', shuffle=True, random_state=42)
print(f"docs: {len(newsgroups.data)}")

max_num_words=10_000 # ordered by term frequency across the corpus
# vectorizer: convert a collection of text documents to a matrix of token counts
# execution time is linear changing max_num_words, as it scans all documents anyway
# complexity = O(N x N_avg) + O(N × V) where N is the number of documents, N_avg = average number of words per document and V is the vocabulary size
with timer("vectorization"):    
    vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=max_num_words, stop_words='english')
    X = vectorizer.fit_transform(newsgroups.data)
print(f"shape: {X.shape}")

# LDA model: Latent Dirichlet Allocation with online variational Bayes algorithm.
# A statistical topic model (NLP technique) that applies unsupervised learning on large text datasets in order to produce a summary set of terms derived from those documents
# LDA explains how a collection of text documents can be described by a set of unobserved "topics." 
# For example, given a set of news articles, LDA might discover that one topic is characterized by words like "president", "government", and "election", while another is characterized by "team", "game", and "score". 
# execution time increases with words, topics to discover, and vocabulary size
# complexity O(I x N × K × V) where I is the number of iterations, N is the number of word tokens, K is the number of topics, and V is the vocabulary size
# each iteration must update topic-word distributions across the entire vocabulary
# only 10_000 most frequent words are considered, not TRILLIONS of tokens!
with timer("fitting"):
    lda = LatentDirichletAllocation(n_components=10, random_state=42, max_iter=50)
    lda.fit(X)

# results visualization
pyLDAvis.enable_notebook()
panel = pyLDAvis.lda_model.prepare(lda, X, vectorizer)
pyLDAvis.display(panel)


start: fetch
end: fetch - exec time: 0.3020 seconds
docs: 18846

start: vectorization
end: vectorization - exec time: 4.1183 seconds
shape: (18846, 10000)

start: fitting
end: fitting - exec time: 328.1398 seconds


### Some analogy with Embeddings

**LDA topics** and **embedding dimensions** both represent compressed, lower-dimensional representations of text:

- **LDA with k topics**: Each document becomes a k-dimensional probability distribution over topics
- **Embeddings with d dimensions**: Each document/word becomes a d-dimensional dense vector

Both are `dimensionality reduction` techniques that map high-dimensional sparse representations (like bag-of-words with vocabulary size V) into lower-dimensional spaces (k topics or d dimensions, where k, d << V).

#### Key differences

1. **Interpretability**: LDA topics are human-interpretable (each topic is a distribution over words). Embedding dimensions are generally not interpretable individually—meaning emerges from the *combination* of dimensions.

2. **Distribution vs. continuous space**: LDA gives probability distributions (values sum to 1). Embeddings give points in continuous vector space.

3. **Sparsity**: LDA document representations are `sparse` (many near-zero topic weights). Embeddings are `dense` (all dimensions typically have non-zero values).

4. **What they capture**: LDA captures thematic structure through word co-occurrence patterns. Embeddings capture semantic relationships, syntax, and context in more nuanced ways.

5. **Dimensionality**:
    - **LDA**: 10-200 topics is typical, depending on corpus size and granularity desired
    - **Embeddings**: 1000+ dimensions, with 768-3072 being common in practice

6. **Ingestion and updating**:
    - **LDA**: Requires retraining to incorporate new documents
      <br/>During LDA training:
      - **Updates topic-word distributions** across entire vocabulary for each iteration
      - **Updates document-topic distributions** for each document
      - Requires **multiple passes** through the corpus (iterative optimization)
      - Must converge through repeated iterations (typically 50-1000+)    

    - **Embeddings**: New documents can be embedded independently without retraining the model
      <br/>During document ingestion with embeddings:
      - **Nothing needs retraining** if using pre-trained models (like BERT)
      - The embedding model parameters are **frozen**
      - Each document is processed **independently** through the neural network
      - Indexing happens at the database level after embeddings are generated
      - Only the **index structure** is updated    

#### Embeddings advantages

- **No iterative training** needed for new documents
- **Parallel processing**: Each document can be embedded independently
- **Consistent**: Same input always produces same embedding
- **Vocabulary size** doesn't affect embedding generation